In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open '$SPARK_VERSION-bin-hadoop2.7.tgz'


Exception: Unable to find py4j in /content/spark-3.1.3-bin-hadoop2.7\python, your SPARK_HOME may not be configured correctly

In [3]:
from pyspark.sql import SparkSession

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.25.jre7.jar

In [ ]:
spark = SparkSession.builder.appName('Amazon Reviews ETL').config('spark.driver.extraClassPath', '/content/postgresql-42.2.25.jre7.jar').getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(''), sep='\t', header=True, inferSchema=True)

In [ ]:
twenty_or_more_df = df.filter(df["total_votes"] >= 20)

In [ ]:
fifty_percent_df = twenty_or_more_df.filter(twenty_or_more_df['helpful_votes'] / twenty_or_more_df['total_votes'] >= 0.5)

In [ ]:
paid_df = fifty_percent_df.filter(fifty_percent_df['vine'] == 'Y')

In [ ]:
not_paid_df = fifty_percent_df.filter(fifty_percent_df['vine'] == 'N')

In [ ]:
fifty_percent_df.show()
twenty_or_more_df.show()
paid_df.show()
not_paid_df.show()

In [ ]:
paid_five_stars = paid_df.select('star_rating').where(paid_df.star_rating == 5).count()

In [ ]:
paid_five_stars

In [ ]:
not_paid_five_stars = not_paid_df.select('star_rating').where(not_paid_df.star_rating ==5).count()

In [ ]:
not_paid_five_stars

In [ ]:
paid_star_percentage = paid_df.select('star_rating').where(paid_df.star_rating == 5).count() / paid_df.select('star_rating').count()

In [ ]:
paid_star_percentage

In [ ]:
not_paid_star_percentage = not_paid_df.select('star_rating').where(not_paid_df.star_rating == 5).count() / not_paid_df.select('star_rating').count()

In [ ]:
not_paid_star_percentage

In [ ]:
vine